In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def get_weather(city):
    city = city.lower().replace(" ", "-")
    url = f"https://www.timeanddate.com/weather/india/{city}"
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve data for {city}. Please check the city name.")
        return None

    soup = BeautifulSoup(response.content, "html.parser")

    try:
        location = soup.find("h1").text.strip()
        temperature = soup.find("div", class_="h2").text.strip()
        weather_desc = soup.find("p").text.strip()
        
        humidity = "N/A"
        for row in soup.find_all("tr"):
            if "Humidity" in row.text:
                td_elements = row.find_all("td")
                if len(td_elements) > 1:
                    humidity = td_elements[1].text.strip()
                break

        # Prepare data dictionary
        data = {
            "City": city.title(),
            "Location": location,
            "Temperature": temperature,
            "Condition": weather_desc,
            "Humidity": humidity,
            "DateTime": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        return data
    except Exception as e:
        print("Error parsing weather data:", e)
        return None

# User input
city_name = input("Enter Indian city name (e.g., Mumbai, Delhi): ")
weather_data = get_weather(city_name)

# Save to CSV
if weather_data:
    df = pd.DataFrame([weather_data])
    file_name = f"{city_name}_weather_log.csv"

    # Append if file exists, otherwise create new
    try:
        existing_df = pd.read_csv(file_name)
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        updated_df = df

    updated_df.to_csv(file_name, index=False)
    print(f"\nWeather data saved to {file_name}")
    print(df.to_string(index=False))
    import shutil
shutil.move(file_name, f"./{file_name}")


Enter Indian city name (e.g., Mumbai, Delhi):  Mumbai



Weather data saved to Mumbai_weather_log.csv
  City                              Location Temperature Condition Humidity            DateTime
Mumbai Weather in Mumbai, Maharashtra, India       28 °C Overcast.      N/A 2025-06-19 16:47:35


'./Mumbai_weather_log.csv'